# Chapter 5: Formatting Output

**Scenario:** You need your AI to talk to another computer program. The program expects neat, clean data (like JSON or a list), but the AI keeps replying with "Sure! Here is the data...". This breaks your code.

In this notebook, you will learn:
- 📦 **Output Tags**: Making the AI wrap its answer in XML tags for easy finding.
- 🤐 **Prefilling**: Starting the sentence for the AI to force a specific format.
- 📋 **JSON Mode**: Tricks to get perfect JSON data.

---

## 1. Setup

Standard setup.

In [1]:
# Install tools
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [3]:
def get_completion(prompt, system_prompt=None, prefill=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    if prefill:
        return prefill + response.choices[0].message.content
    return response.choices[0].message.content

## 2. XML Output Tags

You can ask the model to put its answer inside tags. This makes it very easy to find the answer using code.

In [4]:
prompt = """
Please write a haiku about a Rabbit.
Put it inside <haiku> tags.
"""

print(get_completion(prompt))

<haiku>
Soft fur, twitching nose,
Hops through green, a flash of white,
Then gone, in the grass.
</haiku>



## 3. Prefilling (The "Assistant" Turn)

Prefilling means "putting words in the AI's mouth". You start the answer for it.

**Why?**
1.  **Force Format**: Start with `{` to force JSON.
2.  **Force Tone**: Start with "I am sorry, but" to force a refusal.
3.  **Skip Intro**: Start with the answer directly.

In [5]:
prompt = "Write a haiku about a Cat."
prefill = "<haiku>"

# The AI will see that it has already started with "<haiku>" and will just finish the poem.
print(get_completion(prompt, prefill=prefill))

<haiku>
Soft paws tread so light,
Emerald eyes watch the world,
Purrs a gentle song.
</haiku>



## 4. JSON Format

JSON is the standard format for data. Let's force the AI to write JSON.

In [6]:
prompt = """
Write a haiku about a Cat. 
Use JSON format with keys "first_line", "second_line", "third_line".
"""
prefill = "{"

print(get_completion(prompt, prefill=prefill))

{
  "first_line": "Soft paws tread so light,",
  "second_line": "A purr, a gentle rumble,",
  "third_line": "Sunbeam nap begins."
}



## 5. Exercises

### Exercise 1: Force an Opinion

**Goal:** Force the model to argue that **Cats are better than Dogs**.

Most models are neutral. Use `PREFILL` to start the sentence: "Cats are better than dogs because..."

In [7]:
prompt = "Which is better: Cats or Dogs?"

# Your task: Set this variable to force the model's hand
PREFILL = ""

response = get_completion(prompt, prefill=PREFILL)
print(response)

# Grading
if "Cats are better" in response or "cats are better" in response.lower():
    print("✅ You successfully biased the model!")
else:
    print("❌ Neutral answer? Try a stronger prefill.")

Ah, the age-old question! There's no single "better" answer, as it truly depends on individual preferences and lifestyle. Here's a breakdown to help you decide which might be a better fit for you:

**Dogs:**

*   **Pros:**
    *   **Loyalty and Companionship:** Dogs are often incredibly loyal and form strong bonds with their owners.
    *   **Active Lifestyle:** Dogs encourage you to be more active, requiring walks, playtime, and outdoor adventures.
    *   **Trainability:** Many dog breeds are highly trainable, making them suitable for various activities and tasks.
    *   **Social Interaction:** Dogs can be great for socializing, both for you and for them, at dog parks or during walks.
    *   **Security:** Dogs can provide a sense of security and can be protective of their homes and families.

*   **Cons:**
    *   **Higher Maintenance:** Dogs generally require more time, attention, and money than cats. This includes feeding, grooming, training, vet visits, and potential dog-walking

### Exercise 2: Two Haikus

**Goal:** Ask for TWO haikus. Use `PREFILL` to make sure it skips the introduction and starts right away with `<haiku>`.

In [8]:
prompt = "Write two haikus about Space."
prefill = "[Your prefill here]"

response = get_completion(prompt, prefill=prefill)
print(response)

# Grading
if response.count("<haiku>") >= 2 and response.strip().startswith("<haiku>"):
    print("✅ Good formatting!")
else:
    print("❌ Check your prefill.")

[Your prefill here]

**First haiku**

Stars like diamond dust,
Scattered on black velvet cloth,
Silent, endless night.

**Second haiku**

Distant planets gleam,
Mysteries in cosmic dance,
New worlds wait above.

❌ Check your prefill.


## Summary

1.  **Output formatted data** (JSON/XML) for code.